In [2]:
import copy
import csv
import logging
import sys
import time
import warnings

import importlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
from sklearn.metrics import r2_score

import pypsa
from n_dimensional_datasets import *
from plotter import *

from IPython.display import display # for better Pandas printing 

warnings.filterwarnings('ignore')
logger = logging.getLogger("pypsa")
logger.setLevel("WARNING")
%matplotlib inline

path_to_powerflow_example = "../../pypsa/examples/ieee-13/"
path_to_powerflow_data = path_to_powerflow_example + "/ieee-13-with-load-gen/"
path_to_powerflow_results = path_to_powerflow_data + "results/"

sys.path.append(path_to_powerflow_example)
from ieee13_pf import run

from change_powerflow_data import set_sample_size

def reject_outliers(data, m=3, return_positions=False):
    positions = abs(data - np.mean(data)) < m * np.std(data)
    if return_positions:
        return positions
    return data[positions]

## data sampling

In [3]:
n_original_samples = 2

if n_original_samples < 2:
    raise ValueError("n_original_samples must be an integer >1")

sample_size = 10000

In [4]:
def personalise_column_names(df, name):
        new_columns = []
        for column in df.columns:
            new_columns.append(name +  "-" + str(column))
        df.columns = new_columns
        return pd.DataFrame(df)

def collect_data(data):
    data["loads"] = personalise_column_names(pd.read_csv(path_to_powerflow_data + "loads-p_set.csv"), "load")
    data["vmags"] = personalise_column_names(pd.read_csv(path_to_powerflow_results + "vmags.csv"), "vmag")
    data["vangs"] = personalise_column_names(pd.read_csv(path_to_powerflow_results + "vangs.csv"), "vang")
    data["qmags"] = personalise_column_names(pd.read_csv(path_to_powerflow_results + "qmags.csv"), "qmag")
    data["linemags"] = personalise_column_names(pd.read_csv(path_to_powerflow_results + "linemags.csv"), "linemag")

In [5]:
# data_to_change = ["loads-p_set", "snapshots", "loads-q_set"]

# set_sample_size(path_to_powerflow_data, data_to_change, sample_size, n_original_samples, seed=None)
# network = run() # savefig=["real-loading", ""reactive-loading", "generation", "line-loading", "reactive-feedin"]


data = {"loads": [], "vmags": [], "vangs": [], "qmags": [], "linemags": []}
collect_data(data)

## train

In [6]:
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from scipy.interpolate import LinearNDInterpolator
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, make_scorer

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from custom_transformers import DataFrameSelector, RejectOutliers
from sklearn.decomposition import PCA
from scoring import rmse

In [9]:
features = data["loads"].drop("load-name", axis=1)
labels = data["vmags"].drop(["vmag-name", "vmag-Substation"], axis=1) #loc[:,["vmag-632", "vmag-671", "vmag-675"]]
features_and_labels = features.join(labels)

In [10]:
# data transformations

all_pipeline = Pipeline([
    ("outliers", RejectOutliers(labels.columns)),
])

features_and_labels = pd.DataFrame(all_pipeline.fit_transform(features_and_labels)) 
features = features_and_labels[[col for col in features_and_labels if 'load' in col]]
labels = features_and_labels[[col for col in features_and_labels if 'vmag' in col]]

feature_pipeline = Pipeline([
#     ("selector", DataFrameSelector(["load-671", "load-675"])),
    ("pca", PCA()),
#     ("scaler", StandardScaler()),
])
feature_pipeline.set_params(pca__n_components=0.95)

attributes = list(labels.keys())
attributes.remove("vmag-634")
attributes.remove("vmag-650")
label_pipeline = Pipeline([
    ("selector", DataFrameSelector(attributes))
])

# features = pd.DataFrame(feature_pipeline.fit_transform(features)) #columns=features.columns
labels = pd.DataFrame(label_pipeline.fit_transform(labels), columns=attributes)

In [11]:
training_percentage = 80
n_samples = features.shape[0]
n_training_samples = int(n_samples*(training_percentage/100))

random_seed=0
X_train = features.sample(n_training_samples, random_state=random_seed)
y_train = labels.sample(n_training_samples, random_state=random_seed)
X_val = features[~features.isin(X_train)].dropna()
y_val = labels[~labels.isin(y_train)].dropna()

X_train = X_train.values
y_train = y_train.values
X_val = X_val.values
y_val = y_val.values

In [26]:
##grid search params
grid_svr = {"C": [1, 10], "epsilon": [0.001, 0.0001, 0.00001], "kernel":["linear", "rbf"]}
grid_rf = {"n_estimators": [10, 100, 1000], "oob_score": [False, True]}
grid_ann = {"hidden_layer_sizes": [1, 10, 100], 
            "activation": ["relu", "logistic"],
            "alpha": [0.00001, 0.0001, 0.001, 0.01], 
            "beta_1":[0.09, 0.45, 0.9], 
            "beta_2": [0.0999, 0.45, 0.999]}
grid_params = {"svr": grid_svr, "rf": grid_rf, "ann": grid_ann}

In [ ]:
training_stats = {"n_training_samples": n_training_samples,
                  "n_validation_samples": n_samples-n_training_samples,
                  "n_features": X_train.shape[1],
                  "n_labels": y_train.shape[1],
                  "interp_run": False}
stats = {"trainscore": [], 
         "trainscorevar": [],
         "valscore": [],
         "valscorevar": [],
         "rmse": [],
         "time": []}
approx_type = {"svr": copy.deepcopy(stats),
               "rf": copy.deepcopy(stats),
               "ann": copy.deepcopy(stats),
               "interp": copy.deepcopy(stats)}
approx_type["interp"].pop("trainscorevar")
approx_type["interp"].pop("valscorevar")


time_start = time.time()
### setup approximators


## random forest
forest = RandomForestRegressor()

forest_xval_training_score = reject_outliers(cross_val_score(forest, X_train, y_train, cv=5, n_jobs=-1, scoring=make_scorer(r2_score)))
forest_xval_val_score = reject_outliers(cross_val_score(forest, X_val, y_val, cv=5, n_jobs=-1, scoring=make_scorer(r2_score)))
approx_type["rf"]["trainscore"].append(forest_xval_training_score.mean())
approx_type["rf"]["trainscorevar"].append(forest_xval_training_score.std())
approx_type["rf"]["valscore"].append(forest_xval_val_score.mean())
approx_type["rf"]["valscorevar"].append(forest_xval_val_score.std())

forest.fit(X_train, y_train)
approx_type["rf"]["rmse"].append(rmse(forest.predict(X_val), y_val))

time_forest = time.time()
approx_type["rf"]["time"].append(time_forest-time_start)

grid = GridSearchCV(forest, grid_params["rf"], cv=5, n_jobs=-1, scoring=make_scorer(r2_score), iid=False)
grid.fit(X_train, y_train)
print("\n\n rf grid: \n\n")
display(pd.DataFrame(grid.cv_results_))
                                
## support vector regression
n_labels = y_train.shape[1]
svr = copy.deepcopy(stats)
svr_labels = {"y_train": None, "y_val": None}
for idx in range(n_labels):
    svr_labels["y_train"] = y_train.T[idx].T
    svr_labels["y_val"] = y_val.T[idx].T
    clf = SVR(gamma='scale', C=1.0, epsilon=0.0002, kernel='linear')
    '''
    Scikit-Learn cross-validation features expect a utility function (greater is better) rather than a cost function
    (lower is better), so the scoring function is actually the opposite of the MSE (i.e., a negative value), 
    which is why the preceding code computes -scores before calculating the square root.
    - A. Geron, Hands on Machine Learning pg 101 
    '''
    svr_xval_training_score = reject_outliers(cross_val_score(clf, X_train, svr_labels["y_train"], cv=5, n_jobs=-1, scoring=make_scorer(r2_score)))
    svr_xval_val_score = reject_outliers(cross_val_score(clf, X_val, svr_labels["y_val"], cv=5, n_jobs=-1, scoring=make_scorer(r2_score)))    
    svr["trainscore"].append(svr_xval_training_score.mean())
    svr["trainscorevar"].append(svr_xval_training_score.std())
    svr["valscore"].append(svr_xval_val_score.mean())
    svr["valscorevar"].append(svr_xval_val_score.std())

    clf.fit(X_train, svr_labels["y_train"])
    svr["rmse"].append(rmse(clf.predict(X_val), svr_labels["y_val"]))

    time_svr = time.time()
    svr["time"].append(time_svr - time_forest)
    
    if idx == (n_labels-1):
        display("rmse: ", svr["rmse"])
        label_results = pd.DataFrame(columns=labels.columns)
        label_results.loc[0] = svr["trainscore"]
        display(label_results)

approx_type["svr"]["trainscore"].append(np.mean(svr["trainscore"]))
approx_type["svr"]["trainscorevar"].append(np.mean(svr["trainscorevar"]))
approx_type["svr"]["valscore"].append(np.mean(svr["valscore"]))
approx_type["svr"]["valscorevar"].append(np.mean(svr["valscorevar"]))
approx_type["svr"]["rmse"].append(np.mean(svr["rmse"]))
approx_type["svr"]["time"].append(np.mean(svr["time"]))


## ann
hidden_layer_size = 5
ann = MLPRegressor(hidden_layer_sizes=(hidden_layer_size,),
#                    activation="relu",
                   solver="lbfgs",
                   alpha=0.001,
                   learning_rate_init=0.00000001,
                   tol=1e-6,
                   n_iter_no_change=1000
                  )

ann_xval_training_score = reject_outliers(cross_val_score(ann, X_train, y_train, cv=5, n_jobs=-1, scoring=make_scorer(r2_score)))
ann_xval_val_score = reject_outliers(cross_val_score(ann, X_val, y_val, cv=5, n_jobs=-1, scoring=make_scorer(r2_score)))
approx_type["ann"]["trainscore"].append(ann_xval_training_score.mean())
approx_type["ann"]["trainscorevar"].append(ann_xval_training_score.std())
approx_type["ann"]["valscore"].append(ann_xval_val_score.mean())
approx_type["ann"]["valscorevar"].append(ann_xval_val_score.std())
print("\n\nANN training scores: {}\n\n val scores: {}".format(ann_xval_training_score, ann_xval_val_score))

ann.fit(X_train, y_train)
approx_type["ann"]["rmse"].append(rmse(ann.predict(X_val), y_val))

time_ann = time.time()
approx_type["ann"]["time"].append(time_ann-time_svr)

grid = GridSearchCV(ann, grid_params["ann"], cv=5, n_jobs=-1, scoring=make_scorer(r2_score), iid=False)
grid.fit(X_train, y_train)
print("\n\n ann grid: \n\n")
display(pd.DataFrame(grid.cv_results_))


## interpolation
# interp training gets very slow as the number of features grows
if X_train.shape[1] < 4:
    training_stats["interp_run"] = True
    interp = LinearNDInterpolator(X_train, y_train, fill_value=0)

    time_interp = time.time()
    approx_type["interp"]["time"].append(time_interp-time_ann)

    approx_type["interp"]["trainscore"].append(r2_score(y_train, interp(X_train)))
    approx_type["interp"]["valscore"].append(r2_score(y_val, interp(X_val)))
    approx_type["interp"]["rmse"].append(rmse(interp(X_val), y_val))


## print stats
print("Training Stats\n\n{}".format(pd.DataFrame(training_stats, index=[0])))
for t in approx_type:
    print("\n", t + " Stats: \n")
    display(pd.DataFrame(approx_type[t]).round(3))

print("svr training score - non crossvalidation: ", r2_score(svr_labels["y_train"], clf.predict(X_train)))
print("svr validation score - non crossvalidation: ", r2_score(svr_labels["y_val"], clf.predict(X_val)))
print("rf training score - non crossvalidation: ", r2_score(y_train, forest.predict(X_train)))
print("rf validation score - non crossvalidation: ", r2_score(y_val, forest.predict(X_val)))
print("ann training score - non crossvalidation: ", r2_score(y_train, ann.predict(X_train)))
print("ann validation score - non crossvalidation: ", r2_score(y_val, ann.predict(X_val)))

In [27]:
grid = GridSearchCV(ann, grid_params["ann"], cv=5, n_jobs=-1, scoring=make_scorer(r2_score), iid=False)
grid.fit(X_train, y_train)
print("\n\n ann grid: \n\n")
display(pd.DataFrame(grid.cv_results_))



 ann grid: 




,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_alpha,param_beta_1,param_beta_2,param_hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.110814,0.092080,0.004718,0.002441,relu,1e-05,0.09,0.0999,1,"{'activation': 'relu', 'alpha': 1e-05, 'beta_1...",-0.000679,-0.000955,-0.000197,-0.000285,-0.000036,-0.000430,0.000337,170
1,2.096699,0.310742,0.002508,0.001954,relu,1e-05,0.09,0.0999,10,"{'activation': 'relu', 'alpha': 1e-05, 'beta_1...",0.568394,0.416887,0.516952,0.358329,0.532417,0.478596,0.078363,18
2,10.421671,1.027960,0.019862,0.004948,relu,1e-05,0.09,0.0999,100,"{'activation': 'relu', 'alpha': 1e-05, 'beta_1...",0.542235,0.438345,0.536051,0.564247,0.632739,0.542723,0.062459,5
3,0.336871,0.401705,0.003071,0.001095,relu,1e-05,0.09,0.45,1,"{'activation': 'relu', 'alpha': 1e-05, 'beta_1...",0.419023,-0.000065,-0.000197,-0.000283,0.499855,0.183667,0.226613,116
4,2.074138,0.423886,0.002770,0.002429,relu,1e-05,0.09,0.45,10,"{'activation': 'relu', 'alpha': 1e-05, 'beta_1...",0.358384,0.534117,0.600962,0.586689,0.338011,0.483632,0.112985,17
5,11.251752,2.248055,0.025959,0.005271,relu,1e-05,0.09,0.45,100,"{'activation': 'relu', 'alpha': 1e-05, 'beta_1...",0.455817,0.514723,0.529652,0.596713,0.094028,0.438187,0.177826,34
6,0.084029,0.007519,0.003780,0.000756,relu,1e-05,0.09,0.999,1,"{'activation': 'relu', 'alpha': 1e-05, 'beta_1...",-0.000680,-0.000062,-0.000197,-0.000283,-0.000036,-0.000252,0.000232,162
7,2.443947,0.507667,0.002993,0.002044,relu,1e-05,0.09,0.999,10,"{'activation': 'relu', 'alpha': 1e-05, 'beta_1...",0.621309,0.347340,0.677044,0.174573,0.567382,0.477530,0.188300,20
8,11.777759,3.148471,0.021973,0.006137,relu,1e-05,0.09,0.999,100,"{'activation': 'relu', 'alpha': 1e-05, 'beta_1...",0.589365,0.557053,0.605290,-0.013030,0.632379,0.474211,0.244837,23
9,0.468842,0.537031,0.004223,0.000681,relu,1e-05,0.45,0.0999,1,"{'activation': 'relu', 'alpha': 1e-05, 'beta_1...",-0.000679,-0.000065,0.574802,0.466454,-0.000036,0.208095,0.257472,105


In [31]:
# plotting
from plotter import plot3d_approximation

if X_train.shape[1] == 1:
    plt.plot(X_train, clf.predict(X_train), "x", X_train, y_train, "o")
    plt.show()

    plt.figure()
    plt.plot(X_val, clf.predict(X_val), "x", X_val, y_val, "o")
    plt.show()
    
elif X_train.shape[1] == 2:
    plot3d_approximation(X_train.T, y_train, clf.predict(X_train))

In [32]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(forest, random_state=1).fit(X_val, y_val)
eli5.show_weights(perm) #feature_names=features.columns.tolist()

Weight,Feature
0.8771 ± 0.0362,x5
0.5016 ± 0.0303,x6
0.0516 ± 0.0096,x1
0.0163 ± 0.0072,x8
0.0127 ± 0.0023,x3
0.0107 ± 0.0027,x4
0.0080 ± 0.0044,x7
0.0062 ± 0.0041,x0
0.0027 ± 0.0018,x2


In [ ]:
import pandas as pd
base_path = "/home/ubuntu/Downloads/pypsa/examples/ieee-13/ieee-13-with-load-gen-uniform-data-100000-samples/results/"
foo = pd.read_csv(base_path + "approximating_with_dnn_results-100_samples-2019-09-12-04-36.csv")